## RAG using a pdf book
* see: https://python.langchain.com/docs/use_cases/question_answering/
* using **Cohere** embeddings

In [1]:
# for pdf post processing
import re

# modified to load from Pdf
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# two possible vector store
from langchain.vectorstores import Chroma
from langchain.vectorstores import FAISS

# removed OpenAI, using Cohere embeddings
from langchain.embeddings import CohereEmbeddings

from langchain import hub

# removed OpenAI, using OCI GenAI
from oci.config import from_file

# oci_llm is in a local file
from oci_llm import OCIGenAILLM

from langchain.schema.runnable import RunnablePassthrough

# private configs
from config_private import COMPARTMENT_OCID, COHERE_API_KEY

In [2]:
# to enable some debugging
DEBUG = False

In [3]:
# functions
def get_answer(rag_chain, question):
    response = rag_chain.invoke(question)

    print(f"Question: {question}")
    print("The response:")
    print(response)
    print()

In [4]:
# read OCI config to connect to OCI with API key
CONFIG_PROFILE = "DEFAULT"
config = from_file("~/.oci/config", CONFIG_PROFILE)

# OCI GenAI endpoint (for now Chicago)
ENDPOINT = "https://generativeai.aiservice.us-chicago-1.oci.oraclecloud.com"

# check the config to access to api keys
if DEBUG:
    print(config)

#### Loading the document

In [5]:
# BLOG_POST = "https://python.langchain.com/docs/get_started/introduction"
BOOK = "./oracle-database-23c-new-features-guide.pdf"

loader = PyPDFLoader(BOOK)

data = loader.load()

#### Splitting the document in chunks

In [6]:
# try with smaller chuncks
CHUNK_SIZE = 512
CHUNK_OVERLAP = 50

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP
)

splits = text_splitter.split_documents(data)

In [7]:
print(f"We have splitted the pdf in {len(splits)} splits...")

We have splitted the pdf in 436 splits...


In [8]:
# some post processing

for split in splits:
    split.page_content = split.page_content.replace("\n", " ")
    split.page_content = re.sub("[^a-zA-Z0-9 \n\.]", " ", split.page_content)
    # remove duplicate blank
    split.page_content = " ".join(split.page_content.split())

In [9]:
# have a look at a single split
splits[20].page_content

'Oracle Call Interface OCI Session Pool Statistics 2 24 Oracle Connection Manager in Traffic Director Mode CMAN TDM Support for Direct Path Applications 2 24 Oracle Connection Manager in Traffic Director Mode CMAN TDM Usage Statistics 2 24 Resumable Cursors 2 25 Shut Down Connection Draining for Database Resident Connection Pooling DRCP 2 25 UCP Support for XA Transactions with Sharded Databases 2 25 Database Drivers API Enhancements 2 26 JDBC Support for Database Annotation 2 26'

#### Embeddings and Vectore Store

In [10]:
%%time

# We have substituted OpenAI with HF# see leaderboard here: https://huggingface.co/spaces/mteb/leaderboard
# EMBED_MODEL_NAME = "sentence-transformers/all-mpnet-base-v2"

cohere = CohereEmbeddings(cohere_api_key=COHERE_API_KEY)

# using Chroma or FAISS as Vector store
vectorstore = Chroma.from_documents(documents=splits, embedding=cohere)
# vectorstore = FAISS.from_documents(documents=splits, embedding=hf)

# increased num. of docs to 10 (default to 4)
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

CPU times: user 1.69 s, sys: 255 ms, total: 1.94 s
Wall time: 3.7 s


#### Define the prompt structure

In [11]:
rag_prompt = hub.pull("rlm/rag-prompt")

#### Define the LLM: OCI GenAI

In [17]:
# compartment OCID from config_private.py

llm = OCIGenAILLM(
    temperature=1,
    max_tokens=1500,
    config=config,
    compartment_id=COMPARTMENT_OCID,
    endpoint=ENDPOINT,
    debug=DEBUG,
)

#### Define the (Lang)Chain

In [18]:
rag_chain = {"context": retriever, "question": RunnablePassthrough()} | rag_prompt | llm

#### Process the question

In [19]:
# a list of possible questions
QUESTION1 = "What is the best architecture for an LLM?"
QUESTION2 = "What is LangChain?"
QUESTION3 = "Make a list of database 23c innovations in AI"
QUESTION4 = "List the new features in Oracle Database 23c"
QUESTION5 = "Describe JSON relational duality"
QUESTION6 = "Are there features related to Machine Learning in Oracle Database 23c?"

In [20]:
%%time

# the question
get_answer(rag_chain, question=QUESTION4)

Question: List the new features in Oracle Database 23c
The response:
 Some of the new features in Oracle Database 23c include:
- Up to 4096 columns per table
- Improved machine learning algorithms
- Simplified database migration across platforms using RMAN
- Support for Oracle Database Version Specific RMAN SBT Library
- Blockchain Table User Chains
- Blockchain Table Row Versions
- Resumable Cursors
- Shut Down Connection Draining for Database Resident Connection Pooling DRCP
- UCP Support for XA Transactions with Sharded Databases
- Database Drivers API Enhancements
- JDBC Support for Database Annotation
- CMAN TDM Support for Direct Path Applications
- Oracle Connection Manager in Traffic Director Mode CMAN TDM Usage Statistics
- Selective In Memory Columns
- In Memory Advisor
- OCI and OCCI Password Length Increase
- Updated Kerberos Library and Other Improvements
- Enhancements to RADIUS Configuration
- UTL HTTP Support for SHA 256 and Other Digest Authentication Standards
- XDB H

In [21]:
%%time

# the question
get_answer(rag_chain, question=QUESTION5)

Question: Describe JSON relational duality
The response:
 JSON relational duality is a feature where you can access and update data as either JSON documents or relational tables. This allows developers to take advantage of the strengths of both models, which are simpler and more powerful than Object Relational Mapping (ORM). 

This concept can be explained further by looking at an example. Let's say you have a database that stores customer information in a relational format. You can use JSON relational duality to access this data in a JSON format, which is easier to work with for many developers.

The benefits of using JSON relational duality include:

- Simplifying development: By using a single API to work with both relational and JSON data, you can reduce the complexity of your code and make it more maintainable.
- Enhancing performance: By allowing you to access and update data in a more efficient format, you can improve the performance of your applications.
- Reducing data duplica

In [23]:
%%time

# the question
get_answer(rag_chain, question=QUESTION6)

Question: Are there features related to Machine Learning in Oracle Database 23c?
The response:
 Yes, there are features related to Machine Learning in Oracle Database 23c. Some of these features include:
- Improved Machine Learning Algorithms: New improvements to Oracle In Database Machine Learning algorithms make it simpler to categorize text and data while offering better performance and flexibility.
- Automated Time Series Model Search: This feature enables the Exponential Smoothing algorithm to select the forecasting model type automatically as well as related hyperparameters when you do not.
- Parallel Cross Shard DML Support: This feature allows you to run DML statements in parallel across multiple shards of a table.
- PL SQL Function Cross Shard Query Support: This feature enables you to execute a query in parallel across multiple shards of a table.
- In Memory RAC Level Global Dictionary: This feature provides a global dictionary for in-memory databases, which can be used to st

#### Explore the vectore store

In [24]:
# Retrieve relevant splits for any question using similarity search.

# This is simply "top K" retrieval where we select documents based on embedding similarity to the query.

TOP_K = 10

docs = vectorstore.similarity_search(QUESTION5, k=TOP_K)

len(docs)

10

In [25]:
for i, doc in enumerate(docs):
    print(f"chunk n. {i+1}")
    print(doc.page_content)
    print()

chunk n. 1
2 Application Development JSON JSON Relational Duality JSON Relational Duality Views are fully updatable JSON views over relational data. Data is still stored in relational tables in a highly efficient normalized format but can be accessed by applications in the form of JSON documents. Duality views provide you with game changing flexibility and simplicity by overcoming the historical challenges developers have faced when building applications using relational or document models. Related Resources

chunk n. 2
developer productivity. Note For information about desupported features see Oracle Database Changes Desupports and Deprecations. JSON Relational Duality Data can be transparently accessed and updated as either JSON documents or relational tables. Developers benefit from the strengths of both which are simpler and more powerful than Object Relational Mapping ORM . See JSON Relational Duality . Operational Property Graphs in SQL

chunk n. 3
Contents 1 Introduction 2 Appli